<a href="https://colab.research.google.com/github/njoyce98/HAR-GraphGAN-Dissertation/blob/main/videograph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spektral

     |████████████████████████████████| 122kB 13.5MB/s 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.feature
from sklearn.model_selection import train_test_split
from sklearn import svm
import os
import csv
import time
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from google.colab import drive
import sklearn.metrics as metrics
from tensorflow.keras.callbacks import EarlyStopping
import math
import pickle
import random
import spektral
import copy

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Functions for formatting  headers

def add_xyz(input):
  output = []
  for x in input:
    output.append(x + ".X")
    output.append(x + ".Y")
    output.append(x + ".Z")

  return output

def get_xyz(i):
  output = []
  output.append(i + ".X")
  output.append(i + ".Y")
  output.append(i + ".Z")

  return output

In [ ]:
#Headers 
headers = ['Class', 'Time', 'Hips.X', 'Hips.Y', 'Hips.Z', 'LeftUpLeg.X', 'LeftUpLeg.Y', 'LeftUpLeg.Z', 'LeftUpLegRoll.X',
           'LeftUpLegRoll.Y', 'LeftUpLegRoll.Z', 'LeftLeg.X', 'LeftLeg.Y', 'LeftLeg.Z', 'LeftLegRoll.X', 'LeftLegRoll.Y',
           'LeftLegRoll.Z', 'LeftFoot.X', 'LeftFoot.Y', 'LeftFoot.Z', 'LeftToeBase.X', 'LeftToeBase.Y', 'LeftToeBase.Z',
           'LeftToeBaseEnd.X', 'LeftToeBaseEnd.Y', 'LeftToeBaseEnd.Z', 'RightUpLeg.X', 'RightUpLeg.Y', 'RightUpLeg.Z',
           'RightUpLegRoll.X', 'RightUpLegRoll.Y', 'RightUpLegRoll.Z', 'RightLeg.X', 'RightLeg.Y', 'RightLeg.Z',
           'RightLegRoll.X', 'RightLegRoll.Y', 'RightLegRoll.Z', 'RightFoot.X', 'RightFoot.Y', 'RightFoot.Z',
           'RightToeBase.X', 'RightToeBase.Y', 'RightToeBase.Z', 'RightToeBaseEnd.X', 'RightToeBaseEnd.Y',
           'RightToeBaseEnd.Z', 'spine.X', 'spine.Y', 'spine.Z', 'spine1.X', 'spine1.Y', 'spine1.Z', 'spine2.X', 'spine2.Y',
           'spine2.Z', 'LeftShoulder.X', 'LeftShoulder.Y', 'LeftShoulder.Z', 'LeftArm.X', 'LeftArm.Y', 'LeftArm.Z',
           'LeftArmRoll.X', 'LeftArmRoll.Y', 'LeftArmRoll.Z', 'LeftForeArm.X', 'LeftForeArm.Y', 'LeftForeArm.Z',
           'LeftForeArmRoll.X', 'LeftForeArmRoll.Y', 'LeftForeArmRoll.Z', 'LeftHand.X', 'LeftHand.Y', 'LeftHand.Z',
           'LeftHandEnd.X', 'LeftHandEnd.Y', 'LeftHandEnd.Z', 'RightShoulder.X', 'RightShoulder.Y', 'RightShoulder.Z',
           'RightArm.X', 'RightArm.Y', 'RightArm.Z', 'RightArmRoll.X', 'RightArmRoll.Y', 'RightArmRoll.Z', 'RightForeArm.X',
           'RightForeArm.Y', 'RightForeArm.Z', 'RightForeArmRoll.X', 'RightForeArmRoll.Y', 'RightForeArmRoll.Z', 'RightHand.X',
           'RightHand.Y', 'RightHand.Z', 'RightHandEnd.X', 'RightHandEnd.Y', 'RightHandEnd.Z', 'Neck.X', 'Neck.Y', 'Neck.Z',
           'Head.X', 'Head.Y', 'Head.Z', 'HeadEnd.X', 'HeadEnd.Y', 'HeadEnd.Z']

df = pd.DataFrame(columns = headers)

In [ ]:
# Labels of actions of data
labels = ['Jumping in place', 'Punching', 'Clapping hands', 'Throwing a ball', 'Sit down', 'Stand up']

In [ ]:
action_num = [1, 4, 7, 8, 10, 11]
#8, 10, 11 only 1 repitition
#1,4,7 5 reps

In [ ]:
#Loading in MHAD dataset from drive
path = "/content/drive/My Drive/Colab Notebooks/Datasets/Berkeley/Skeletal"

for subject in range (1,13):
  if (subject < 10):
    subject = "0" + str(subject)
  else:
    subject = str(subject)
  # Cycle through the actions of subject 1
  for a, action in enumerate(action_num):
      div5 = True
      if (action > 7):
        div5 = False
      label = labels[a]
      if (action < 10):
          action = "0" + str(action)
      else:
          action = str(action)
      for repition in range (1,6):
          repition = "0" + str(repition)
          filename = 'skl_s' + subject + '_a' + action + '_r' + repition + "_worldpos.csv"
          # Handle each file and assign label
          my_data = []

          fullpath = path + '/' + filename
          try:
            with open(fullpath, newline='') as csvfile:
                csvread = csv.reader(csvfile, delimiter=',')
                for i, row in enumerate(csvread):
                    if(i != 0):
                        my_row = []
                        my_row.append(label)
                        for col in row:
                                my_row.append(float(col))
                        my_data.append(my_row)

            new_df = pd.DataFrame(data = my_data, columns = headers)
            # If sample has 5 repitions divide sample by 5
            if (div5):
              split = (int)(len(new_df)/5)
              new_df = new_df.iloc[:split]
            df = pd.concat([df, new_df.sample(30).sort_index()], axis = 0)
          except:
            print(filename + " does not exist")

skl_s04_a08_r05_worldpos.csv does not exist


In [ ]:
#Save dataframe to pickle
# with open('/content/drive/My Drive/Colab Notebooks/Datasets/mhad_loaded_data.pickle', 'wb') as output:
#     pickle.dump(df, output)

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Datasets/mhad_loaded_data.pickle', 'rb') as data:
    df = pickle.load(data)

In [ ]:
df

,Class,Time,Hips.X,Hips.Y,Hips.Z,LeftUpLeg.X,LeftUpLeg.Y,LeftUpLeg.Z,LeftLeg.X,LeftLeg.Y,LeftLeg.Z,LeftFoot.X,LeftFoot.Y,LeftFoot.Z,LeftToeBaseEnd.X,LeftToeBaseEnd.Y,LeftToeBaseEnd.Z,RightUpLeg.X,RightUpLeg.Y,RightUpLeg.Z,RightLeg.X,RightLeg.Y,RightLeg.Z,RightFoot.X,RightFoot.Y,RightFoot.Z,RightToeBaseEnd.X,RightToeBaseEnd.Y,RightToeBaseEnd.Z,spine2.X,spine2.Y,spine2.Z,LeftShoulder.X,LeftShoulder.Y,LeftShoulder.Z,LeftForeArm.X,LeftForeArm.Y,LeftForeArm.Z,LeftHand.X,LeftHand.Y,LeftHand.Z,LeftHandEnd.X,LeftHandEnd.Y,LeftHandEnd.Z,RightShoulder.X,RightShoulder.Y,RightShoulder.Z,RightForeArm.X,RightForeArm.Y,RightForeArm.Z,RightHand.X,RightHand.Y,RightHand.Z,RightHandEnd.X,RightHandEnd.Y,RightHandEnd.Z,Neck.X,Neck.Y,Neck.Z,HeadEnd.X,HeadEnd.Y,HeadEnd.Z
23,Jumping in place,0.047917,53.6659,85.5579,10.07540,59.284804,84.386694,9.074974,60.571383,47.268066,10.191593,60.939109,9.884354,4.671864,62.953560,2.738860,24.369830,47.958543,84.652407,9.333229,46.445482,47.569668,11.143132,46.721198,9.951907,7.541377,45.694901,2.819640,27.326786,54.485756,109.900816,10.117037,58.735170,119.176199,9.920577,72.553470,103.327551,16.261376,71.647407,83.401645,23.479425,71.647407,83.401645,23.479425,51.343585,119.608337,9.976879,35.780492,102.512991,15.706108,37.642001,83.017338,23.855786,37.642001,83.017338,23.855786,55.814493,132.380906,12.017108,55.247530,156.407034,12.030313
39,Jumping in place,0.081250,53.7282,85.4758,9.67095,59.352188,84.336368,8.662432,60.486977,47.236850,10.403877,60.912790,9.896114,4.604502,62.721076,2.728987,24.313840,48.024773,84.566567,8.903571,46.542662,47.520325,11.368116,46.790766,9.917942,7.607289,45.774200,2.699239,27.363214,54.491273,109.816496,10.145239,58.729817,119.098920,10.158678,72.495455,103.290723,16.125990,71.578290,83.356129,23.318607,71.578290,83.356129,23.318607,51.337617,119.521091,10.209480,35.862453,102.403077,15.731552,37.765180,82.867555,23.775553,37.765180,82.867555,23.775553,55.788973,132.251585,12.535423,55.255340,156.280140,12.779762
137,Jumping in place,0.285416,54.0002,81.2528,5.68347,59.650630,80.372395,4.568939,59.237447,45.697421,17.916978,60.815196,10.518760,4.201400,61.972288,2.568608,23.628483,48.319608,80.366261,4.740297,47.666989,46.194519,19.319763,47.108123,10.269219,7.606307,46.018967,2.539013,27.109921,54.391474,105.173527,10.272330,58.564856,114.227438,12.449120,72.592018,98.327407,15.290237,71.524344,77.920694,20.981375,71.524344,77.920694,20.981375,51.170658,114.615218,12.419341,36.055215,97.237374,15.340533,38.101900,76.984436,21.306478,38.101900,76.984436,21.306478,55.481187,126.516596,17.621898,54.744421,150.294727,19.141500
155,Jumping in place,0.322916,53.8357,79.4735,4.55964,59.473890,78.603948,3.376699,59.661130,45.215696,19.682656,60.461110,10.622084,4.490993,62.255582,2.503069,23.759352,48.144202,78.663242,3.613603,47.885496,45.685691,20.734025,47.206232,10.319876,7.432061,46.202241,2.504004,26.887872,54.381466,103.159626,10.223763,58.584941,112.072615,12.872965,72.650447,96.111890,15.310136,71.985131,75.531464,20.405858,71.985131,75.531464,20.405858,51.191636,112.477653,12.893631,36.062674,95.055361,14.761138,38.223453,74.650147,20.138986,38.223453,74.650147,20.138986,55.550723,124.069073,18.718355,54.750287,147.786140,19.986328
160,Jumping in place,0.333333,53.8295,78.9006,4.25467,59.464733,78.040315,3.051043,59.801879,45.062334,20.169273,60.433794,10.644859,4.574958,62.375771,2.486118,23.801445,48.135549,78.107190,3.309111,47.980087,45.512803,21.149414,47.267017,10.318599,7.401537,46.263722,2.482568,26.845742,54.400417,102.509641,10.229429,58.606840,111.381365,13.009245,72.683643,95.382127,15.343723,72.073208,74.750401,20.234757,72.073208,74.750401,20.234757,51.213491,111.784024,13.050453,36.076260,94.356804,14.611719,38.275209,73.910581,19.815518,38.275209,73.910581,19.815518,55.581003,123.284363,19.046877,54.757737,146.994042,20.214603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
#position of joints needed to be dropped in MHAD
drop_joints = [2, 4, 6, 9, 11, 13, 15, 16, 19, 20, 22, 26, 27, 29, 33]

In [ ]:
#Calculate headers without joints
new_headers = headers.copy()

for x in drop_joints:
  colx = headers[2 + (x*3)]
  coly = headers[3 + (x*3)]
  colz = headers[4 + (x*3)]
  new_headers.remove(colx)
  new_headers.remove(coly)
  new_headers.remove(colz)
  df = df.drop([colx], axis = 1)
  df = df.drop([coly], axis = 1)
  df = df.drop([colz], axis = 1)

KeyError: ignored

In [ ]:
#Count of each action in samples, must be balanced
df['Class'].value_counts()

Jumping in place    1800
Punching            1800
Clapping hands      1800
Sit down            1800
Stand up            1800
Throwing a ball     1770
Name: Class, dtype: int64

In [ ]:
# Save time column for later use in other files
# with open('/content/drive/My Drive/Colab Notebooks/Datasets/mhad_time.pickle', 'wb') as output:
#     pickle.dump(df['Time'], output)

In [ ]:
#Create new balanced dataframe
df2 = df.drop(['Time'], axis = 1).copy()

label_dfs = []
for x in labels:
    label_dfs.append(df[df2['Class'] == x].head(1770).copy())

balanced_df = pd.DataFrame()
for x in label_dfs:
    balanced_df = pd.concat([balanced_df, x], axis = 0)

In [ ]:
# Resulting balanced classes
balanced_df['Class'].value_counts()

Punching            1770
Stand up            1770
Clapping hands      1770
Throwing a ball     1770
Jumping in place    1770
Sit down            1770
Name: Class, dtype: int64

In [ ]:
# Encode labels
lab_enc = LabelEncoder()
balanced_df['Label'] = lab_enc.fit_transform(balanced_df['Class'])
balanced_df.head()

,Class,Time,Hips.X,Hips.Y,Hips.Z,LeftUpLeg.X,LeftUpLeg.Y,LeftUpLeg.Z,LeftLeg.X,LeftLeg.Y,LeftLeg.Z,LeftFoot.X,LeftFoot.Y,LeftFoot.Z,LeftToeBaseEnd.X,LeftToeBaseEnd.Y,LeftToeBaseEnd.Z,RightUpLeg.X,RightUpLeg.Y,RightUpLeg.Z,RightLeg.X,RightLeg.Y,RightLeg.Z,RightFoot.X,RightFoot.Y,RightFoot.Z,RightToeBaseEnd.X,RightToeBaseEnd.Y,RightToeBaseEnd.Z,spine2.X,spine2.Y,spine2.Z,LeftShoulder.X,LeftShoulder.Y,LeftShoulder.Z,LeftForeArm.X,LeftForeArm.Y,LeftForeArm.Z,LeftHand.X,LeftHand.Y,LeftHand.Z,LeftHandEnd.X,LeftHandEnd.Y,LeftHandEnd.Z,RightShoulder.X,RightShoulder.Y,RightShoulder.Z,RightForeArm.X,RightForeArm.Y,RightForeArm.Z,RightHand.X,RightHand.Y,RightHand.Z,RightHandEnd.X,RightHandEnd.Y,RightHandEnd.Z,Neck.X,Neck.Y,Neck.Z,HeadEnd.X,HeadEnd.Y,HeadEnd.Z,Label
23,Jumping in place,0.047917,53.6659,85.5579,10.07540,59.284804,84.386694,9.074974,60.571383,47.268066,10.191593,60.939109,9.884354,4.671864,62.953560,2.738860,24.369830,47.958543,84.652407,9.333229,46.445482,47.569668,11.143132,46.721198,9.951907,7.541377,45.694901,2.819640,27.326786,54.485756,109.900816,10.117037,58.735170,119.176199,9.920577,72.553470,103.327551,16.261376,71.647407,83.401645,23.479425,71.647407,83.401645,23.479425,51.343585,119.608337,9.976879,35.780492,102.512991,15.706108,37.642001,83.017338,23.855786,37.642001,83.017338,23.855786,55.814493,132.380906,12.017108,55.247530,156.407034,12.030313,1
39,Jumping in place,0.081250,53.7282,85.4758,9.67095,59.352188,84.336368,8.662432,60.486977,47.236850,10.403877,60.912790,9.896114,4.604502,62.721076,2.728987,24.313840,48.024773,84.566567,8.903571,46.542662,47.520325,11.368116,46.790766,9.917942,7.607289,45.774200,2.699239,27.363214,54.491273,109.816496,10.145239,58.729817,119.098920,10.158678,72.495455,103.290723,16.125990,71.578290,83.356129,23.318607,71.578290,83.356129,23.318607,51.337617,119.521091,10.209480,35.862453,102.403077,15.731552,37.765180,82.867555,23.775553,37.765180,82.867555,23.775553,55.788973,132.251585,12.535423,55.255340,156.280140,12.779762,1
137,Jumping in place,0.285416,54.0002,81.2528,5.68347,59.650630,80.372395,4.568939,59.237447,45.697421,17.916978,60.815196,10.518760,4.201400,61.972288,2.568608,23.628483,48.319608,80.366261,4.740297,47.666989,46.194519,19.319763,47.108123,10.269219,7.606307,46.018967,2.539013,27.109921,54.391474,105.173527,10.272330,58.564856,114.227438,12.449120,72.592018,98.327407,15.290237,71.524344,77.920694,20.981375,71.524344,77.920694,20.981375,51.170658,114.615218,12.419341,36.055215,97.237374,15.340533,38.101900,76.984436,21.306478,38.101900,76.984436,21.306478,55.481187,126.516596,17.621898,54.744421,150.294727,19.141500,1
155,Jumping in place,0.322916,53.8357,79.4735,4.55964,59.473890,78.603948,3.376699,59.661130,45.215696,19.682656,60.461110,10.622084,4.490993,62.255582,2.503069,23.759352,48.144202,78.663242,3.613603,47.885496,45.685691,20.734025,47.206232,10.319876,7.432061,46.202241,2.504004,26.887872,54.381466,103.159626,10.223763,58.584941,112.072615,12.872965,72.650447,96.111890,15.310136,71.985131,75.531464,20.405858,71.985131,75.531464,20.405858,51.191636,112.477653,12.893631,36.062674,95.055361,14.761138,38.223453,74.650147,20.138986,38.223453,74.650147,20.138986,55.550723,124.069073,18.718355,54.750287,147.786140,19.986328,1
160,Jumping in place,0.333333,53.8295,78.9006,4.25467,59.464733,78.040315,3.051043,59.801879,45.062334,20.169273,60.433794,10.644859,4.574958,62.375771,2.486118,23.801445,48.135549,78.107190,3.309111,47.980087,45.512803,21.149414,47.267017,10.318599,7.401537,46.263722,2.482568,26.845742,54.400417,102.509641,10.229429,58.606840,111.381365,13.009245,72.683643,95.382127,15.343723,72.073208,74.750401,20.234757,72.073208,74.750401,20.234757,51.213491,111.784024,13.050453,36.076260,94.356804,14.611719,38.275209,73.910581,19.815518,38.275209,73.910581,19.815518,55.581003,123.284363,19.046877,54.757737,146.994042,20.214603,1


In [ ]:
lab_enc.classes_

array(['Clapping hands', 'Jumping in place', 'Punching', 'Sit down',
       'Stand up', 'Throwing a ball'], dtype=object)

In [ ]:
X = balanced_df[new_headers[1:]]
Y = balanced_df['Label']

In [ ]:
# Standard scale the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

scaled_X = pd.DataFrame(data = X, columns = new_headers[1:])
scaled_X['Label'] = Y.values

In [ ]:
# Scaler saved from unscaling in generation
with open('/content/drive/My Drive/Colab Notebooks/Datasets/MHAD_scaler.pickle', 'wb') as output:
    pickle.dump(scaler, output)

In [ ]:
scaled_X

,Time,Hips.X,Hips.Y,Hips.Z,LeftUpLeg.X,LeftUpLeg.Y,LeftUpLeg.Z,LeftLeg.X,LeftLeg.Y,LeftLeg.Z,LeftFoot.X,LeftFoot.Y,LeftFoot.Z,LeftToeBaseEnd.X,LeftToeBaseEnd.Y,LeftToeBaseEnd.Z,RightUpLeg.X,RightUpLeg.Y,RightUpLeg.Z,RightLeg.X,RightLeg.Y,RightLeg.Z,RightFoot.X,RightFoot.Y,RightFoot.Z,RightToeBaseEnd.X,RightToeBaseEnd.Y,RightToeBaseEnd.Z,spine2.X,spine2.Y,spine2.Z,LeftShoulder.X,LeftShoulder.Y,LeftShoulder.Z,LeftForeArm.X,LeftForeArm.Y,LeftForeArm.Z,LeftHand.X,LeftHand.Y,LeftHand.Z,LeftHandEnd.X,LeftHandEnd.Y,LeftHandEnd.Z,RightShoulder.X,RightShoulder.Y,RightShoulder.Z,RightForeArm.X,RightForeArm.Y,RightForeArm.Z,RightHand.X,RightHand.Y,RightHand.Z,RightHandEnd.X,RightHandEnd.Y,RightHandEnd.Z,Neck.X,Neck.Y,Neck.Z,HeadEnd.X,HeadEnd.Y,HeadEnd.Z,Label
0,-1.127255,0.304295,0.052201,0.047522,0.278234,0.054556,0.104367,-0.218424,-0.957696,-0.649890,-0.237028,-0.558396,-0.662466,-0.476398,-0.498540,-0.568335,0.341352,0.068302,-0.002592,0.470570,-0.871130,-0.697535,0.798194,-0.547940,-0.582590,0.558913,-0.435592,-0.544548,0.370511,-0.054944,-0.004934,0.461689,-0.086883,-0.037728,0.290518,-0.023102,0.141227,0.267330,-0.350959,-0.072748,0.267330,-0.350959,-0.072748,0.402314,-0.055479,-0.096925,0.009445,-0.147387,-0.238476,-0.180664,-0.483169,-0.370861,-0.180664,-0.483169,-0.370861,0.461415,-0.077119,-0.129711,0.299816,-0.084963,-0.212044,1
1,-1.119635,0.306131,0.051373,0.040911,0.280338,0.054094,0.097623,-0.221058,-0.958190,-0.646869,-0.236987,-0.556719,-0.663400,-0.480132,-0.496967,-0.568979,0.343148,0.067438,-0.009424,0.473478,-0.872187,-0.694424,0.801014,-0.548341,-0.581855,0.561189,-0.442145,-0.544333,0.371094,-0.055775,-0.004666,0.462320,-0.087693,-0.034331,0.288834,-0.023046,0.139355,0.266117,-0.351054,-0.074991,0.266117,-0.351054,-0.074991,0.402885,-0.056285,-0.093778,0.010481,-0.148190,-0.238414,-0.179506,-0.484114,-0.372508,-0.179506,-0.484114,-0.372508,0.461840,-0.078447,-0.123052,0.301109,-0.086205,-0.204297,1
2,-1.104393,0.310210,0.048070,0.026202,0.284978,0.051606,0.082514,-0.227120,-0.963221,-0.638548,-0.237260,-0.556470,-0.665661,-0.488087,-0.498623,-0.570781,0.347119,0.064013,-0.024476,0.479595,-0.878825,-0.686781,0.805977,-0.553589,-0.581005,0.565417,-0.459107,-0.544505,0.372062,-0.059026,-0.004775,0.463142,-0.090832,-0.027993,0.285987,-0.024581,0.135240,0.263521,-0.352589,-0.080208,0.263521,-0.352589,-0.080208,0.403620,-0.059498,-0.087762,0.012551,-0.151211,-0.239108,-0.177017,-0.487135,-0.377203,-0.177017,-0.487135,-0.377203,0.462090,-0.082648,-0.109781,0.302712,-0.090170,-0.188778,1
3,-1.058668,0.325755,0.032633,-0.023394,0.302340,0.039808,0.029924,-0.253641,-0.977984,-0.578923,-0.245740,-0.535010,-0.668048,-0.517862,-0.507107,-0.574684,0.362071,0.047608,-0.074038,0.500920,-0.906048,-0.639184,0.814659,-0.567559,-0.577941,0.578061,-0.485757,-0.543870,0.369420,-0.074569,-0.002893,0.455426,-0.105181,-0.007577,0.284853,-0.039289,0.126221,0.252206,-0.365731,-0.096835,0.252206,-0.365731,-0.096835,0.396427,-0.075797,-0.065417,0.016707,-0.166710,-0.241313,-0.169193,-0.501647,-0.396150,-0.169193,-0.501647,-0.396150,0.449833,-0.101122,-0.066523,0.289400,-0.108135,-0.137308,1
4,-0.997702,0.338954,-0.011233,-0.097968,0.316895,-0.001485,-0.047188,-0.307120,-1.034531,-0.460923,-0.244936,-0.502046,-0.677229,-0.542966,-0.513225,-0.585938,0.374618,0.004559,-0.149482,0.538825,-0.960643,-0.518581,0.832716,-0.565148,-0.583465,0.600257,-0.489895,-0.549524,0.373427,-0.118768,-0.004259,0.453786,-0.148430,0.023720,0.289633,-0.083264,0.110304,0.243375,-0.403328,-0.128901,0.243375,-0.403328,-0.128901,0.394725,-0.121243,-0.033875,0.023699,-0.209639,-0.243832,-0.160588,-0.538561,-0.429092,-0.160588,-0.538561,-0.429092,0.441210,-0.150952,0.001487,0.277126,-0.156120,-0.053891,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10615,1.484132,0.5072

In [ ]:
# Headers of NTU Dataset
ntu_headers = ['Pelvis.X', 'Pelvis.Y', 'Pelvis.Z', 'Abs.X', 'Abs.Y', 'Abs.Z', 'Neck.X', 'Neck.Y', 'Neck.Z', 'Head.X', 'Head.Y', 'Head.Z',
               'Left shoulder.X', 'Left shoulder.Y', 'Left shoulder.Z', 'Left elbow.X', 'Left elbow.Y', 'Left elbow.Z', 'Left forearm.X',
               'Left forearm.Y', 'Left forearm.Z', 'Right shoulder.X', 'Right shoulder.Y', 'Right shoulder.Z', 'Right elbow.X', 'Right elbow.Y', 
               'Right elbow.Z', 'Right forearm.X', 'Right forearm.Y', 'Right forearm.Z', 'Left hip.X', 'Left hip.Y', 'Left hip.Z', 'Left knee.X', 
               'Left knee.Y', 'Left knee.Z', 'Left ankle.X', 'Left ankle.Y', 'Left ankle.Z', 'Left toes.X', 'Left toes.Y', 'Left toes.Z', 
               'Right hip.X', 'Right hip.Y', 'Right hip.Z', 'Right knee.X', 'Right knee.Y', 'Right knee.Z', 'Right ankle.X', 'Right ankle.Y', 
               'Right ankle.Z', 'Right toes.X', 'Right toes.Y', 'Right toes.Z', 'Left fingers.X', 'Left fingers.Y', 'Left fingers.Z', 'Right fingers.X', 
               'Right fingers.Y', 'Right fingers.Z', 'Label', 'Time']

In [ ]:
# Headers of MHAD dataset lined up to match NTU
ntuheaders_noXYZ = ['Hips', 'spine2', 'Neck', 'HeadEnd', 'LeftShoulder', 'LeftForeArm', 'LeftHand', 'RightShoulder',
                    'RightForeArm', 'RightHand', 'LeftUpLeg', 'LeftLeg', 'LeftFoot', 'LeftToeBaseEnd', 'RightUpLeg',
                    'RightLeg', 'RightFoot', 'RightToeBaseEnd', 'LeftHandEnd', 'RightHandEnd']

arranged_headers = add_xyz(ntuheaders_noXYZ)
arranged_headers.append('Label')
arranged_headers.append('Time')

arranged_X = scaled_X.copy()

# Re organise MHAD headers to fit NTU headers
arranged_X = arranged_X[arranged_headers]
# Replace MHAD headers with NTU headers
arranged_X.columns = ntu_headers

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Datasets/MHAD_df30fps.pickle', 'wb') as output:
    pickle.dump(arranged_X, output)

#NTURGBD

In [ ]:
#Actions to be loaded from NTU
ntu_actions = [50, 27, 10, 9, 8, 7]

In [ ]:
#Path to dataset
ntu_path = "/content/drive/My Drive/Colab Notebooks/Datasets/NTU-RGBD/Skeletal/"

ntu_joints = ['Pelvis', 'Abs', 'Neck', 'Head', 'Left shoulder', 'Left elbow', 'Left forearm', 'Left wrist', 'Right shoulder',
               'Right elbow', 'Right forearm', 'Right wrist', 'Left hip', 'Left knee', 'Left ankle', 'Left toes',
               'Right hip', 'Right knee', 'Right ankle', 'Right toes', 'Torso', 'Left fingers', 'Left thumb', 
               'Right fingers', 'Right thumb']

ntu_df = pd.DataFrame()

In [ ]:
ntu_headers = add_xyz(ntu_joints)
print(ntu_headers)

['Pelvis.X', 'Pelvis.Y', 'Pelvis.Z', 'Abs.X', 'Abs.Y', 'Abs.Z', 'Neck.X', 'Neck.Y', 'Neck.Z', 'Head.X', 'Head.Y', 'Head.Z', 'Left shoulder.X', 'Left shoulder.Y', 'Left shoulder.Z', 'Left elbow.X', 'Left elbow.Y', 'Left elbow.Z', 'Left forearm.X', 'Left forearm.Y', 'Left forearm.Z', 'Left wrist.X', 'Left wrist.Y', 'Left wrist.Z', 'Right shoulder.X', 'Right shoulder.Y', 'Right shoulder.Z', 'Right elbow.X', 'Right elbow.Y', 'Right elbow.Z', 'Right forearm.X', 'Right forearm.Y', 'Right forearm.Z', 'Right wrist.X', 'Right wrist.Y', 'Right wrist.Z', 'Left hip.X', 'Left hip.Y', 'Left hip.Z', 'Left knee.X', 'Left knee.Y', 'Left knee.Z', 'Left ankle.X', 'Left ankle.Y', 'Left ankle.Z', 'Left toes.X', 'Left toes.Y', 'Left toes.Z', 'Right hip.X', 'Right hip.Y', 'Right hip.Z', 'Right knee.X', 'Right knee.Y', 'Right knee.Z', 'Right ankle.X', 'Right ankle.Y', 'Right ankle.Z', 'Right toes.X', 'Right toes.Y', 'Right toes.Z', 'Torso.X', 'Torso.Y', 'Torso.Z', 'Left fingers.X', 'Left fingers.Y', 'Left fin

In [ ]:
# Functions for parsing filenames
def add_zeros(n):
  if (n < 10):
    n = '0' + str(n)
  else:
    n = str(n)
  
  return n

def get_action_label(a):
    switcher = {
        50: 'Punching',
        27: 'Jumping in place',
        10: 'Clapping hands',
        9: 'Stand up',
        8: 'Sit down',
        7: 'Throwing a ball'
    }
    return switcher.get(a)

In [ ]:
# Creates generated time stamps (shot in 30fps)
def make_times(length):
  output = []
  count = 0.0
  for x in range(length):
    output.append(count)
    count += (1/30)
  return output

In [ ]:
#Load NTU RGB-D
for S in range(1, 18):
    S = add_zeros(S)
    for C in range(1, 4):
        C = add_zeros(C)
        for P in range(1, 40):
            P = add_zeros(P)
            for R in range(1, 2):
                R = add_zeros(R)
                for A in ntu_actions:
                    label = get_action_label(A)
                    A = add_zeros(A)
                    filename = "S0" + S + "C0" + C + "P0" + P + "R0" + R + "A0" + A + ".skeleton.npy"
                    print(filename)
                    try:
                        ntu_data = np.load(ntu_path+filename ,allow_pickle=True).item()
                        skel_data = ntu_data['skel_body0']
                        length, w, d = skel_data.shape
                        skel_data = skel_data.reshape(length, 75)
                        new_df = pd.DataFrame(skel_data, columns = ntu_headers)
                        new_df.insert(0,'Class',label)
                        times = make_times(length)
                        new_df.insert(1,'Time', times)

                        ntu_df = pd.concat([ntu_df, new_df.sample(30).sort_index()], axis = 0)
                    except:
                        print(filename + " does not exist")

Streaming output truncated to the last 5000 lines.
S014C001P005R001A007.skeleton.npy does not exist
S014C001P006R001A050.skeleton.npy
S014C001P006R001A050.skeleton.npy does not exist
S014C001P006R001A027.skeleton.npy
S014C001P006R001A027.skeleton.npy does not exist
S014C001P006R001A010.skeleton.npy
S014C001P006R001A010.skeleton.npy does not exist
S014C001P006R001A009.skeleton.npy
S014C001P006R001A009.skeleton.npy does not exist
S014C001P006R001A008.skeleton.npy
S014C001P006R001A008.skeleton.npy does not exist
S014C001P006R001A007.skeleton.npy
S014C001P006R001A007.skeleton.npy does not exist
S014C001P007R001A050.skeleton.npy
S014C001P007R001A027.skeleton.npy
S014C001P007R001A010.skeleton.npy
S014C001P007R001A009.skeleton.npy
S014C001P007R001A008.skeleton.npy
S014C001P007R001A007.skeleton.npy
S014C001P008R001A050.skeleton.npy
S014C001P008R001A027.skeleton.npy
S014C001P008R001A010.skeleton.npy
S014C001P008R001A009.skeleton.npy
S014C001P008R001A008.skeleton.npy
S014C001P008R001A007.skeleto

In [ ]:
ntu_df

,Class,Time,Pelvis.X,Pelvis.Y,Pelvis.Z,Abs.X,Abs.Y,Abs.Z,Neck.X,Neck.Y,Neck.Z,Head.X,Head.Y,Head.Z,Left shoulder.X,Left shoulder.Y,Left shoulder.Z,Left elbow.X,Left elbow.Y,Left elbow.Z,Left forearm.X,Left forearm.Y,Left forearm.Z,Left wrist.X,Left wrist.Y,Left wrist.Z,Right shoulder.X,Right shoulder.Y,Right shoulder.Z,Right elbow.X,Right elbow.Y,Right elbow.Z,Right forearm.X,Right forearm.Y,Right forearm.Z,Right wrist.X,Right wrist.Y,Right wrist.Z,Left hip.X,Left hip.Y,Left hip.Z,Left knee.X,Left knee.Y,Left knee.Z,Left ankle.X,Left ankle.Y,Left ankle.Z,Left toes.X,Left toes.Y,Left toes.Z,Right hip.X,Right hip.Y,Right hip.Z,Right knee.X,Right knee.Y,Right knee.Z,Right ankle.X,Right ankle.Y,Right ankle.Z,Right toes.X,Right toes.Y,Right toes.Z,Torso.X,Torso.Y,Torso.Z,Left fingers.X,Left fingers.Y,Left fingers.Z,Left thumb.X,Left thumb.Y,Left thumb.Z,Right fingers.X,Right fingers.Y,Right fingers.Z,Right thumb.X,Right thumb.Y,Right thumb.Z
1,Punching,0.033333,-0.851461,0.190473,3.385317,-0.873675,0.405992,3.341556,-0.893416,0.616289,3.288781,-0.887586,0.720652,3.304067,-0.904962,0.490093,3.197685,-0.892435,0.287543,3.192696,-0.830740,0.092745,3.253016,-0.824895,0.054532,3.272867,-0.852048,0.546703,3.398313,-0.866330,0.340471,3.208597,-0.871337,0.140511,3.222293,-0.852559,0.105367,3.250268,-0.855232,0.173840,3.307511,-0.885213,-0.223930,3.448230,-0.919281,-0.582173,3.556920,-0.884843,-0.641926,3.558505,-0.833915,0.204473,3.409895,-1.010324,-0.054227,3.326061,-0.994694,-0.375400,3.425643,-0.908884,-0.411212,3.390626,-0.888867,0.564512,3.303554,-0.803544,-0.002269,3.275172,-0.803977,0.055234,3.268072,-0.844396,0.066380,3.253638,-0.862835,0.100521,3.242068
4,Punching,0.133333,-0.855053,0.187868,3.384807,-0.875003,0.404237,3.341107,-0.893196,0.615918,3.288976,-0.888233,0.721866,3.303422,-0.904869,0.490349,3.197587,-0.892783,0.287779,3.192587,-0.831808,0.093138,3.251619,-0.824900,0.053943,3.271977,-0.851829,0.546333,3.398508,-0.863415,0.338717,3.210150,-0.868014,0.139046,3.226354,-0.851837,0.102882,3.250480,-0.858167,0.172507,3.305798,-0.885664,-0.224512,3.447908,-0.918813,-0.582745,3.556034,-0.884657,-0.642873,3.556962,-0.836595,0.201908,3.413122,-1.010453,-0.054964,3.315553,-0.993322,-0.375379,3.417104,-0.907580,-0.411312,3.382157,-0.888851,0.563951,3.303622,-0.803245,-0.002273,3.273920,-0.804311,0.055154,3.272744,-0.840264,0.066403,3.254639,-0.861313,0.095787,3.242273
7,Punching,0.233333,-0.854513,0.190540,3.386178,-0.874996,0.405459,3.341802,-0.893447,0.616655,3.289410,-0.889143,0.721986,3.302921,-0.906619,0.490863,3.199082,-0.894861,0.288254,3.193356,-0.832398,0.092883,3.252568,-0.826221,0.054786,3.271179,-0.852080,0.547069,3.398944,-0.865013,0.339707,3.210407,-0.869819,0.139867,3.225180,-0.851894,0.101897,3.250625,-0.858284,0.174820,3.305789,-0.885535,-0.221979,3.447829,-0.919358,-0.582590,3.555666,-0.834573,-0.617275,3.520064,-0.836634,0.204934,3.416338,-1.007676,-0.055253,3.324959,-0.991277,-0.376852,3.423126,-0.905712,-0.412905,3.387551,-0.889126,0.564644,3.304131,-0.807319,-0.001935,3.287656,-0.806250,0.060204,3.269745,-0.844600,0.066516,3.260261,-0.862634,0.100689,3.240673
8,Punching,0.266667,-0.853835,0.192235,3.385366,-0.874882,0.406064,3.341729,-0.893495,0.616790,3.289571,-0.889225,0.722005,3.302799,-0.907587,0.491384,3.198921,-0.894804,0.288817,3.193162,-0.833265,0.093285,3.252112,-0.826398,0.053701,3.272075,-0.852128,0.547205,3.399104,-0.865845,0.341446,3.208809,-0.870796,0.141466,3.222360,-0.853774,0.103176,3.246106,-0.858181,0.176730,3.307045,-0.885519,-0.220595,3.447711,-0.918365,-0.583467,3.553069,-0.884613,-0.642641,3.556574,-0.834594,0.207305,3.410215,-1.007208,-0.054106,3.329581,-0.990252,-0.375902,3.427088,-0.904757,-0.411988,3.391226,-0.889157,0.564822,3.304237,-0.804762,-0.002270,3.280167,-0.807366,0.063570,3.270121,-0.844574,0.066396,3.254457,-0.866644,0.102743,3.235545
12,Punching,0.400000,-0.855853,0.191656,3.384125,-0.876142,0.405174,3.340672,-0.895017,0.614866,3.288297,-0.890203,0.720538,3.299983,-0.90724

In [ ]:
# #save dataset as pickle
# with open('/content/drive/My Drive/Colab Notebooks/Datasets/ntu_loaded_data.pickle', 'wb') as output:
#     pickle.dump(ntu_df, output)

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Datasets/ntu_loaded_data.pickle', 'rb') as data:
    ntu_df = pickle.load(data)

In [ ]:
ntu_df

,Class,Time,Pelvis.X,Pelvis.Y,Pelvis.Z,Abs.X,Abs.Y,Abs.Z,Neck.X,Neck.Y,Neck.Z,Head.X,Head.Y,Head.Z,Left shoulder.X,Left shoulder.Y,Left shoulder.Z,Left elbow.X,Left elbow.Y,Left elbow.Z,Left forearm.X,Left forearm.Y,Left forearm.Z,Left wrist.X,Left wrist.Y,Left wrist.Z,Right shoulder.X,Right shoulder.Y,Right shoulder.Z,Right elbow.X,Right elbow.Y,Right elbow.Z,Right forearm.X,Right forearm.Y,Right forearm.Z,Right wrist.X,Right wrist.Y,Right wrist.Z,Left hip.X,Left hip.Y,Left hip.Z,Left knee.X,Left knee.Y,Left knee.Z,Left ankle.X,Left ankle.Y,Left ankle.Z,Left toes.X,Left toes.Y,Left toes.Z,Right hip.X,Right hip.Y,Right hip.Z,Right knee.X,Right knee.Y,Right knee.Z,Right ankle.X,Right ankle.Y,Right ankle.Z,Right toes.X,Right toes.Y,Right toes.Z,Torso.X,Torso.Y,Torso.Z,Left fingers.X,Left fingers.Y,Left fingers.Z,Left thumb.X,Left thumb.Y,Left thumb.Z,Right fingers.X,Right fingers.Y,Right fingers.Z,Right thumb.X,Right thumb.Y,Right thumb.Z
1,Punching,0.033333,-0.851461,0.190473,3.385317,-0.873675,0.405992,3.341556,-0.893416,0.616289,3.288781,-0.887586,0.720652,3.304067,-0.904962,0.490093,3.197685,-0.892435,0.287543,3.192696,-0.830740,0.092745,3.253016,-0.824895,0.054532,3.272867,-0.852048,0.546703,3.398313,-0.866330,0.340471,3.208597,-0.871337,0.140511,3.222293,-0.852559,0.105367,3.250268,-0.855232,0.173840,3.307511,-0.885213,-0.223930,3.448230,-0.919281,-0.582173,3.556920,-0.884843,-0.641926,3.558505,-0.833915,0.204473,3.409895,-1.010324,-0.054227,3.326061,-0.994694,-0.375400,3.425643,-0.908884,-0.411212,3.390626,-0.888867,0.564512,3.303554,-0.803544,-0.002269,3.275172,-0.803977,0.055234,3.268072,-0.844396,0.066380,3.253638,-0.862835,0.100521,3.242068
4,Punching,0.133333,-0.855053,0.187868,3.384807,-0.875003,0.404237,3.341107,-0.893196,0.615918,3.288976,-0.888233,0.721866,3.303422,-0.904869,0.490349,3.197587,-0.892783,0.287779,3.192587,-0.831808,0.093138,3.251619,-0.824900,0.053943,3.271977,-0.851829,0.546333,3.398508,-0.863415,0.338717,3.210150,-0.868014,0.139046,3.226354,-0.851837,0.102882,3.250480,-0.858167,0.172507,3.305798,-0.885664,-0.224512,3.447908,-0.918813,-0.582745,3.556034,-0.884657,-0.642873,3.556962,-0.836595,0.201908,3.413122,-1.010453,-0.054964,3.315553,-0.993322,-0.375379,3.417104,-0.907580,-0.411312,3.382157,-0.888851,0.563951,3.303622,-0.803245,-0.002273,3.273920,-0.804311,0.055154,3.272744,-0.840264,0.066403,3.254639,-0.861313,0.095787,3.242273
7,Punching,0.233333,-0.854513,0.190540,3.386178,-0.874996,0.405459,3.341802,-0.893447,0.616655,3.289410,-0.889143,0.721986,3.302921,-0.906619,0.490863,3.199082,-0.894861,0.288254,3.193356,-0.832398,0.092883,3.252568,-0.826221,0.054786,3.271179,-0.852080,0.547069,3.398944,-0.865013,0.339707,3.210407,-0.869819,0.139867,3.225180,-0.851894,0.101897,3.250625,-0.858284,0.174820,3.305789,-0.885535,-0.221979,3.447829,-0.919358,-0.582590,3.555666,-0.834573,-0.617275,3.520064,-0.836634,0.204934,3.416338,-1.007676,-0.055253,3.324959,-0.991277,-0.376852,3.423126,-0.905712,-0.412905,3.387551,-0.889126,0.564644,3.304131,-0.807319,-0.001935,3.287656,-0.806250,0.060204,3.269745,-0.844600,0.066516,3.260261,-0.862634,0.100689,3.240673
8,Punching,0.266667,-0.853835,0.192235,3.385366,-0.874882,0.406064,3.341729,-0.893495,0.616790,3.289571,-0.889225,0.722005,3.302799,-0.907587,0.491384,3.198921,-0.894804,0.288817,3.193162,-0.833265,0.093285,3.252112,-0.826398,0.053701,3.272075,-0.852128,0.547205,3.399104,-0.865845,0.341446,3.208809,-0.870796,0.141466,3.222360,-0.853774,0.103176,3.246106,-0.858181,0.176730,3.307045,-0.885519,-0.220595,3.447711,-0.918365,-0.583467,3.553069,-0.884613,-0.642641,3.556574,-0.834594,0.207305,3.410215,-1.007208,-0.054106,3.329581,-0.990252,-0.375902,3.427088,-0.904757,-0.411988,3.391226,-0.889157,0.564822,3.304237,-0.804762,-0.002270,3.280167,-0.807366,0.063570,3.270121,-0.844574,0.066396,3.254457,-0.866644,0.102743,3.235545
12,Punching,0.400000,-0.855853,0.191656,3.384125,-0.876142,0.405174,3.340672,-0.895017,0.614866,3.288297,-0.890203,0.720538,3.299983,-0.90724

In [ ]:
# Joints to drop from NTU
ntu_drop_joints = ['Right thumb', 'Left thumb', 'Torso', 'Left wrist', 'Right wrist']

ntu_new_headers = ntu_headers.copy()

drop_headers = add_xyz(ntu_drop_joints)

for x in drop_headers:
  ntu_new_headers.remove(x)
  ntu_df = ntu_df.drop([x], axis = 1)

In [ ]:
ntu_df['Class'].value_counts()

Throwing a ball     14130
Jumping in place    14130
Sit down            14070
Punching            14070
Clapping hands      14010
Stand up            13920
Name: Class, dtype: int64

In [ ]:
# Balance dataset
ntu_label_dfs = []
for x in ntu_actions:
    x = get_action_label(x)
    print(x)
    ntu_label_dfs.append(ntu_df[ntu_df['Class'] == x].head(13920).copy())

Punching
Jumping in place
Clapping hands
Stand up
Sit down
Throwing a ball


In [ ]:
ntu_balanced_df = pd.DataFrame()
for x in ntu_label_dfs:
    ntu_balanced_df = pd.concat([ntu_balanced_df, x], axis = 0)

# Encode labels
lab_enc = LabelEncoder()

ntu_balanced_df['Label'] = lab_enc.fit_transform(ntu_balanced_df['Class'])
ntu_balanced_df.head()

,Class,Time,Pelvis.X,Pelvis.Y,Pelvis.Z,Abs.X,Abs.Y,Abs.Z,Neck.X,Neck.Y,Neck.Z,Head.X,Head.Y,Head.Z,Left shoulder.X,Left shoulder.Y,Left shoulder.Z,Left elbow.X,Left elbow.Y,Left elbow.Z,Left forearm.X,Left forearm.Y,Left forearm.Z,Right shoulder.X,Right shoulder.Y,Right shoulder.Z,Right elbow.X,Right elbow.Y,Right elbow.Z,Right forearm.X,Right forearm.Y,Right forearm.Z,Left hip.X,Left hip.Y,Left hip.Z,Left knee.X,Left knee.Y,Left knee.Z,Left ankle.X,Left ankle.Y,Left ankle.Z,Left toes.X,Left toes.Y,Left toes.Z,Right hip.X,Right hip.Y,Right hip.Z,Right knee.X,Right knee.Y,Right knee.Z,Right ankle.X,Right ankle.Y,Right ankle.Z,Right toes.X,Right toes.Y,Right toes.Z,Left fingers.X,Left fingers.Y,Left fingers.Z,Right fingers.X,Right fingers.Y,Right fingers.Z,Label
1,Punching,0.033333,-0.851461,0.190473,3.385317,-0.873675,0.405992,3.341556,-0.893416,0.616289,3.288781,-0.887586,0.720652,3.304067,-0.904962,0.490093,3.197685,-0.892435,0.287543,3.192696,-0.830740,0.092745,3.253016,-0.852048,0.546703,3.398313,-0.866330,0.340471,3.208597,-0.871337,0.140511,3.222293,-0.855232,0.173840,3.307511,-0.885213,-0.223930,3.448230,-0.919281,-0.582173,3.556920,-0.884843,-0.641926,3.558505,-0.833915,0.204473,3.409895,-1.010324,-0.054227,3.326061,-0.994694,-0.375400,3.425643,-0.908884,-0.411212,3.390626,-0.803544,-0.002269,3.275172,-0.844396,0.066380,3.253638,2
4,Punching,0.133333,-0.855053,0.187868,3.384807,-0.875003,0.404237,3.341107,-0.893196,0.615918,3.288976,-0.888233,0.721866,3.303422,-0.904869,0.490349,3.197587,-0.892783,0.287779,3.192587,-0.831808,0.093138,3.251619,-0.851829,0.546333,3.398508,-0.863415,0.338717,3.210150,-0.868014,0.139046,3.226354,-0.858167,0.172507,3.305798,-0.885664,-0.224512,3.447908,-0.918813,-0.582745,3.556034,-0.884657,-0.642873,3.556962,-0.836595,0.201908,3.413122,-1.010453,-0.054964,3.315553,-0.993322,-0.375379,3.417104,-0.907580,-0.411312,3.382157,-0.803245,-0.002273,3.273920,-0.840264,0.066403,3.254639,2
7,Punching,0.233333,-0.854513,0.190540,3.386178,-0.874996,0.405459,3.341802,-0.893447,0.616655,3.289410,-0.889143,0.721986,3.302921,-0.906619,0.490863,3.199082,-0.894861,0.288254,3.193356,-0.832398,0.092883,3.252568,-0.852080,0.547069,3.398944,-0.865013,0.339707,3.210407,-0.869819,0.139867,3.225180,-0.858284,0.174820,3.305789,-0.885535,-0.221979,3.447829,-0.919358,-0.582590,3.555666,-0.834573,-0.617275,3.520064,-0.836634,0.204934,3.416338,-1.007676,-0.055253,3.324959,-0.991277,-0.376852,3.423126,-0.905712,-0.412905,3.387551,-0.807319,-0.001935,3.287656,-0.844600,0.066516,3.260261,2
8,Punching,0.266667,-0.853835,0.192235,3.385366,-0.874882,0.406064,3.341729,-0.893495,0.616790,3.289571,-0.889225,0.722005,3.302799,-0.907587,0.491384,3.198921,-0.894804,0.288817,3.193162,-0.833265,0.093285,3.252112,-0.852128,0.547205,3.399104,-0.865845,0.341446,3.208809,-0.870796,0.141466,3.222360,-0.858181,0.176730,3.307045,-0.885519,-0.220595,3.447711,-0.918365,-0.583467,3.553069,-0.884613,-0.642641,3.556574,-0.834594,0.207305,3.410215,-1.007208,-0.054106,3.329581,-0.990252,-0.375902,3.427088,-0.904757,-0.411988,3.391226,-0.804762,-0.002270,3.280167,-0.844574,0.066396,3.254457,2
12,Punching,0.400000,-0.855853,0.191656,3.384125,-0.876142,0.405174,3.340672,-0.895017,0.614866,3.288297,-0.890203,0.720538,3.299983,-0.907246,0.489395,3.195759,-0.896733,0.286831,3.192298,-0.833174,0.094535,3.249568,-0.853648,0.545284,3.397833,-0.866696,0.339283,3.207767,-0.871585,0.139269,3.220982,-0.859712,0.177300,3.315540,-0.885953,-0.222633,3.449489,-0.920351,-0.576387,3.549401,-0.886499,-0.635752,3.552494,-0.838320,0.201868,3.398692,-1.010931,-0.061800,3.333943,-0.992500,-0.383003,3.432954,-0.906819,-0.418849,3.397931,-0.804190,-0.002259,3.277660,-0.843457,0.066305,3.249961,2


In [ ]:
ntu_balanced_Y = ntu_balanced_df['Label']

In [ ]:
lab_enc_name_mapping = dict(zip(lab_enc.classes_, lab_enc.transform(lab_enc.classes_)))
print(lab_enc_name_mapping)

{'Clapping hands': 0, 'Jumping in place': 1, 'Punching': 2, 'Sit down': 3, 'Stand up': 4, 'Throwing a ball': 5}


In [ ]:
ntu_new_headers.append('Label')
ntu_new_headers.append('Time')

ntu_balanced_X = ntu_balanced_df[ntu_new_headers]
ntu_Y = ntu_balanced_df['Label']

In [ ]:
# Scale Time
ntu_balanced_X[['Time', 'Label']] = scaler.fit_transform(ntu_balanced_X[['Time', 'Label']])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [ ]:
#Re assign label to unscaled labels
ntu_balanced_X['Label'] = ntu_balanced_Y

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Datasets/NTU_scaler.pickle', 'wb') as output:
    pickle.dump(scaler, output)

In [ ]:
ntu_balanced_X

,Pelvis.X,Pelvis.Y,Pelvis.Z,Abs.X,Abs.Y,Abs.Z,Neck.X,Neck.Y,Neck.Z,Head.X,Head.Y,Head.Z,Left shoulder.X,Left shoulder.Y,Left shoulder.Z,Left elbow.X,Left elbow.Y,Left elbow.Z,Left forearm.X,Left forearm.Y,Left forearm.Z,Right shoulder.X,Right shoulder.Y,Right shoulder.Z,Right elbow.X,Right elbow.Y,Right elbow.Z,Right forearm.X,Right forearm.Y,Right forearm.Z,Left hip.X,Left hip.Y,Left hip.Z,Left knee.X,Left knee.Y,Left knee.Z,Left ankle.X,Left ankle.Y,Left ankle.Z,Left toes.X,Left toes.Y,Left toes.Z,Right hip.X,Right hip.Y,Right hip.Z,Right knee.X,Right knee.Y,Right knee.Z,Right ankle.X,Right ankle.Y,Right ankle.Z,Right toes.X,Right toes.Y,Right toes.Z,Left fingers.X,Left fingers.Y,Left fingers.Z,Right fingers.X,Right fingers.Y,Right fingers.Z,Label,Time
1,-0.851461,0.190473,3.385317,-0.873675,0.405992,3.341556,-0.893416,0.616289,3.288781,-0.887586,0.720652,3.304067,-0.904962,0.490093,3.197685,-0.892435,0.287543,3.192696,-0.830740,0.092745,3.253016,-0.852048,0.546703,3.398313,-0.866330,0.340471,3.208597,-0.871337,0.140511,3.222293,-0.855232,0.173840,3.307511,-0.885213,-0.223930,3.448230,-0.919281,-0.582173,3.556920,-0.884843,-0.641926,3.558505,-0.833915,0.204473,3.409895,-1.010324,-0.054227,3.326061,-0.994694,-0.375400,3.425643,-0.908884,-0.411212,3.390626,-0.803544,-0.002269,3.275172,-0.844396,0.066380,3.253638,2,-1.535114
3,-0.856917,0.186912,3.384592,-0.875302,0.403990,3.340795,-0.893065,0.615745,3.288331,-0.887377,0.719484,3.303063,-0.905162,0.490336,3.196429,-0.892661,0.287831,3.192625,-0.831800,0.093088,3.251542,-0.851697,0.546159,3.397862,-0.875663,0.386976,3.166765,-0.881683,0.186821,3.179677,-0.859662,0.171652,3.306313,-0.885915,-0.225554,3.448463,-0.919018,-0.582647,3.556357,-0.884089,-0.641653,3.559124,-0.837719,0.201504,3.412772,-1.009691,-0.057696,3.318765,-0.993724,-0.378556,3.419133,-0.908039,-0.414572,3.384128,-0.806529,-0.002713,3.271068,-0.858690,0.112785,3.230285,2,-1.436817
8,-0.853835,0.192235,3.385366,-0.874882,0.406064,3.341729,-0.893495,0.616790,3.289571,-0.889225,0.722005,3.302799,-0.907587,0.491384,3.198921,-0.894804,0.288817,3.193162,-0.833265,0.093285,3.252112,-0.852128,0.547205,3.399104,-0.865845,0.341446,3.208809,-0.870796,0.141466,3.222360,-0.858181,0.176730,3.307045,-0.885519,-0.220595,3.447711,-0.918365,-0.583467,3.553069,-0.884613,-0.642641,3.556574,-0.834594,0.207305,3.410215,-1.007208,-0.054106,3.329581,-0.990252,-0.375902,3.427088,-0.904757,-0.411988,3.391226,-0.804762,-0.002270,3.280167,-0.844574,0.066396,3.254457,2,-1.191076
9,-0.857510,0.194379,3.384274,-0.875864,0.406873,3.341603,-0.893476,0.616906,3.289706,-0.889368,0.722049,3.302572,-0.907655,0.491370,3.198743,-0.895469,0.288820,3.194168,-0.833432,0.093321,3.252358,-0.852109,0.547321,3.399240,-0.866757,0.342365,3.208127,-0.871874,0.142266,3.220559,-0.862824,0.180954,3.310925,-0.886075,-0.218409,3.447556,-0.918804,-0.583771,3.553812,-0.883419,-0.644118,3.552774,-0.840890,0.207718,3.402912,-1.006214,-0.060576,3.334605,-0.989013,-0.382318,3.432169,-0.903593,-0.418474,3.396355,-0.806381,-0.002135,3.285547,-0.848390,0.075197,3.252127,2,-1.141928
11,-0.856387,0.193400,3.384281,-0.875891,0.406342,3.341192,-0.894204,0.615992,3.289247,-0.890000,0.720951,3.300788,-0.906717,0.489650,3.196755,-0.896034,0.287081,3.193052,-0.832526,0.094014,3.249890,-0.852836,0.546408,3.398782,-0.866942,0.341041,3.208088,-0.872027,0.140881,3.219909,-0.860137,0.179096,3.313187,-0.885881,-0.220409,3.448537,-0.918562,-0.584519,3.554486,-0.882652,-0.643864,3.555355,-0.839770,0.205990,3.402364,-1.006841,-0.060874,3.333135,-0.988609,-0.382155,3.431928,-0.902997,-0.418077,3.396786,-0.805232,-0.002178,3.281205,-0.842058,0.066196,3.244628,2,-1.043632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,-0.028689,0.006032,3.545196,-0.038324,0.285783,3.386380,-0.057220,0.555792

In [ ]:
# save dataframe as pickle
with open('/content/drive/My Drive/Colab Notebooks/Datasets/NTU_df30fps.pickle', 'wb') as output:
    pickle.dump(ntu_balanced_X, output)

#Loaded in df

In [ ]:
#load dataframes
with open('/content/drive/My Drive/Colab Notebooks/Datasets/MHAD_df30fps.pickle', 'rb') as data:
    mhad_df = pickle.load(data)

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Datasets/NTU_df30fps.pickle', 'rb') as data:
    ntu_df = pickle.load(data)

In [ ]:
mhad_df

,Pelvis.X,Pelvis.Y,Pelvis.Z,Abs.X,Abs.Y,Abs.Z,Neck.X,Neck.Y,Neck.Z,Head.X,Head.Y,Head.Z,Left shoulder.X,Left shoulder.Y,Left shoulder.Z,Left elbow.X,Left elbow.Y,Left elbow.Z,Left forearm.X,Left forearm.Y,Left forearm.Z,Right shoulder.X,Right shoulder.Y,Right shoulder.Z,Right elbow.X,Right elbow.Y,Right elbow.Z,Right forearm.X,Right forearm.Y,Right forearm.Z,Left hip.X,Left hip.Y,Left hip.Z,Left knee.X,Left knee.Y,Left knee.Z,Left ankle.X,Left ankle.Y,Left ankle.Z,Left toes.X,Left toes.Y,Left toes.Z,Right hip.X,Right hip.Y,Right hip.Z,Right knee.X,Right knee.Y,Right knee.Z,Right ankle.X,Right ankle.Y,Right ankle.Z,Right toes.X,Right toes.Y,Right toes.Z,Left fingers.X,Left fingers.Y,Left fingers.Z,Right fingers.X,Right fingers.Y,Right fingers.Z,Label,Time
0,0.304295,0.052201,0.047522,0.370511,-0.054944,-0.004934,0.461415,-0.077119,-0.129711,0.299816,-0.084963,-0.212044,0.461689,-0.086883,-0.037728,0.290518,-0.023102,0.141227,0.267330,-0.350959,-0.072748,0.402314,-0.055479,-0.096925,0.009445,-0.147387,-0.238476,-0.180664,-0.483169,-0.370861,0.278234,0.054556,0.104367,-0.218424,-0.957696,-0.649890,-0.237028,-0.558396,-0.662466,-0.476398,-0.498540,-0.568335,0.341352,0.068302,-0.002592,0.470570,-0.871130,-0.697535,0.798194,-0.547940,-0.582590,0.558913,-0.435592,-0.544548,0.267330,-0.350959,-0.072748,-0.180664,-0.483169,-0.370861,1,-1.127255
1,0.306131,0.051373,0.040911,0.371094,-0.055775,-0.004666,0.461840,-0.078447,-0.123052,0.301109,-0.086205,-0.204297,0.462320,-0.087693,-0.034331,0.288834,-0.023046,0.139355,0.266117,-0.351054,-0.074991,0.402885,-0.056285,-0.093778,0.010481,-0.148190,-0.238414,-0.179506,-0.484114,-0.372508,0.280338,0.054094,0.097623,-0.221058,-0.958190,-0.646869,-0.236987,-0.556719,-0.663400,-0.480132,-0.496967,-0.568979,0.343148,0.067438,-0.009424,0.473478,-0.872187,-0.694424,0.801014,-0.548341,-0.581855,0.561189,-0.442145,-0.544333,0.266117,-0.351054,-0.074991,-0.179506,-0.484114,-0.372508,1,-1.119635
2,0.310210,0.048070,0.026202,0.372062,-0.059026,-0.004775,0.462090,-0.082648,-0.109781,0.302712,-0.090170,-0.188778,0.463142,-0.090832,-0.027993,0.285987,-0.024581,0.135240,0.263521,-0.352589,-0.080208,0.403620,-0.059498,-0.087762,0.012551,-0.151211,-0.239108,-0.177017,-0.487135,-0.377203,0.284978,0.051606,0.082514,-0.227120,-0.963221,-0.638548,-0.237260,-0.556470,-0.665661,-0.488087,-0.498623,-0.570781,0.347119,0.064013,-0.024476,0.479595,-0.878825,-0.686781,0.805977,-0.553589,-0.581005,0.565417,-0.459107,-0.544505,0.263521,-0.352589,-0.080208,-0.177017,-0.487135,-0.377203,1,-1.104393
3,0.325755,0.032633,-0.023394,0.369420,-0.074569,-0.002893,0.449833,-0.101122,-0.066523,0.289400,-0.108135,-0.137308,0.455426,-0.105181,-0.007577,0.284853,-0.039289,0.126221,0.252206,-0.365731,-0.096835,0.396427,-0.075797,-0.065417,0.016707,-0.166710,-0.241313,-0.169193,-0.501647,-0.396150,0.302340,0.039808,0.029924,-0.253641,-0.977984,-0.578923,-0.245740,-0.535010,-0.668048,-0.517862,-0.507107,-0.574684,0.362071,0.047608,-0.074038,0.500920,-0.906048,-0.639184,0.814659,-0.567559,-0.577941,0.578061,-0.485757,-0.543870,0.252206,-0.365731,-0.096835,-0.169193,-0.501647,-0.396150,1,-1.058668
4,0.338954,-0.011233,-0.097968,0.373427,-0.118768,-0.004259,0.441210,-0.150952,0.001487,0.277126,-0.156120,-0.053891,0.453786,-0.148430,0.023720,0.289633,-0.083264,0.110304,0.243375,-0.403328,-0.128901,0.394725,-0.121243,-0.033875,0.023699,-0.209639,-0.243832,-0.160588,-0.538561,-0.429092,0.316895,-0.001485,-0.047188,-0.307120,-1.034531,-0.460923,-0.244936,-0.502046,-0.677229,-0.542966,-0.513225,-0.585938,0.374618,0.004559,-0.149482,0.538825,-0.960643,-0.518581,0.832716,-0.565148,-0.583465,0.600257,-0.489895,-0.549524,0.243375,-0.403328,-0.128901,-0.160588,-0.538561,-0.429092,1,-0.997702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10615,0.507252,

In [ ]:
ntu_df

,Pelvis.X,Pelvis.Y,Pelvis.Z,Abs.X,Abs.Y,Abs.Z,Neck.X,Neck.Y,Neck.Z,Head.X,Head.Y,Head.Z,Left shoulder.X,Left shoulder.Y,Left shoulder.Z,Left elbow.X,Left elbow.Y,Left elbow.Z,Left forearm.X,Left forearm.Y,Left forearm.Z,Right shoulder.X,Right shoulder.Y,Right shoulder.Z,Right elbow.X,Right elbow.Y,Right elbow.Z,Right forearm.X,Right forearm.Y,Right forearm.Z,Left hip.X,Left hip.Y,Left hip.Z,Left knee.X,Left knee.Y,Left knee.Z,Left ankle.X,Left ankle.Y,Left ankle.Z,Left toes.X,Left toes.Y,Left toes.Z,Right hip.X,Right hip.Y,Right hip.Z,Right knee.X,Right knee.Y,Right knee.Z,Right ankle.X,Right ankle.Y,Right ankle.Z,Right toes.X,Right toes.Y,Right toes.Z,Left fingers.X,Left fingers.Y,Left fingers.Z,Right fingers.X,Right fingers.Y,Right fingers.Z,Label,Time
2,-0.859557,0.184862,3.384113,-0.875499,0.403873,3.340841,-0.893056,0.615834,3.288471,-0.887510,0.720322,3.303248,-0.904303,0.489852,3.197232,-0.892436,0.287290,3.192683,-0.830713,0.092715,3.252954,-0.851688,0.546249,3.398002,-0.866653,0.340460,3.207849,-0.871719,0.140536,3.221853,-0.861891,0.169678,3.310357,-0.886472,-0.228687,3.450389,-0.919171,-0.582431,3.556499,-0.884244,-0.641640,3.558827,-0.845452,0.195870,3.403242,-1.016294,-0.068096,3.327986,-1.000832,-0.389605,3.426449,-0.915103,-0.425601,3.391709,-0.803425,-0.002271,3.274672,-0.843460,0.066305,3.249971,2,0.066667
5,-0.854726,0.188004,3.385394,-0.875061,0.404446,3.341635,-0.893587,0.616351,3.289617,-0.888319,0.722265,3.304060,-0.905675,0.490316,3.199133,-0.893942,0.287694,3.193002,-0.832408,0.093462,3.252444,-0.852219,0.546765,3.399149,-0.865439,0.339223,3.210843,-0.870275,0.139389,3.225647,-0.858107,0.172772,3.306807,-0.885552,-0.224195,3.448821,-0.918386,-0.582923,3.556569,-0.885293,-0.643046,3.558569,-0.836384,0.201830,3.413194,-1.011180,-0.054397,3.316031,-0.993631,-0.374768,3.417669,-0.907939,-0.410739,3.382470,-0.808244,-0.001983,3.286722,-0.844621,0.066403,3.254804,2,0.166667
6,-0.854356,0.190044,3.385878,-0.874808,0.405301,3.341562,-0.893268,0.616729,3.289213,-0.888987,0.722248,3.303648,-0.906281,0.490717,3.198998,-0.893634,0.288117,3.192302,-0.832415,0.092959,3.252535,-0.851900,0.547143,3.398746,-0.865335,0.339595,3.210464,-0.870194,0.139764,3.225290,-0.858124,0.174540,3.306451,-0.885419,-0.222522,3.448098,-0.918372,-0.583064,3.556691,-0.895996,-0.615234,3.611328,-0.835977,0.203650,3.414159,-1.009260,-0.055101,3.322970,-0.992740,-0.376280,3.422372,-0.907151,-0.412330,3.386833,-0.807710,-0.002718,3.275866,-0.842674,0.066243,3.246968,2,0.200000
7,-0.854513,0.190540,3.386178,-0.874996,0.405459,3.341802,-0.893447,0.616655,3.289410,-0.889143,0.721986,3.302921,-0.906619,0.490863,3.199082,-0.894861,0.288254,3.193356,-0.832398,0.092883,3.252568,-0.852080,0.547069,3.398944,-0.865013,0.339707,3.210407,-0.869819,0.139867,3.225180,-0.858284,0.174820,3.305789,-0.885535,-0.221979,3.447829,-0.919358,-0.582590,3.555666,-0.834573,-0.617275,3.520064,-0.836634,0.204934,3.416338,-1.007676,-0.055253,3.324959,-0.991277,-0.376852,3.423126,-0.905712,-0.412905,3.387551,-0.807319,-0.001935,3.287656,-0.844600,0.066516,3.260261,2,0.233333
8,-0.853835,0.192235,3.385366,-0.874882,0.406064,3.341729,-0.893495,0.616790,3.289571,-0.889225,0.722005,3.302799,-0.907587,0.491384,3.198921,-0.894804,0.288817,3.193162,-0.833265,0.093285,3.252112,-0.852128,0.547205,3.399104,-0.865845,0.341446,3.208809,-0.870796,0.141466,3.222360,-0.858181,0.176730,3.307045,-0.885519,-0.220595,3.447711,-0.918365,-0.583467,3.553069,-0.884613,-0.642641,3.556574,-0.834594,0.207305,3.410215,-1.007208,-0.054106,3.329581,-0.990252,-0.375902,3.427088,-0.904757,-0.411988,3.391226,-0.804762,-0.002270,3.280167,-0.844574,0.066396,3.254457,2,0.266667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,-0.031740,0.014671,3.532804,-0.028042,0.296925,3.385236,-0.030835,0.571998,3.228

#Convert to graphs

In [ ]:
# Calculate bones through connected joints
ntu_joints = ['Pelvis', 'Abs', 'Neck', 'Head', 'Left shoulder', 'Left elbow', 'Left forearm', 'Right shoulder',
 'Right elbow', 'Right forearm', 'Left hip', 'Left knee', 'Left ankle', 'Left toes', 'Right hip',
 'Right knee', 'Right ankle', 'Right toes', 'Left fingers', 'Right fingers']

start_nodes = ["Pelvis", "Abs", "Neck", "Abs", "Left shoulder", "Left elbow", "Left forearm", "Abs", "Right shoulder", "Right elbow", "Right forearm",
               "Pelvis", "Left hip", "Left knee", "Left ankle", "Pelvis", "Right hip", "Right knee", "Right ankle"]

end_nodes = ["Abs", "Neck", "Head", "Left shoulder", "Left elbow", "Left forearm", "Left fingers", "Right shoulder", "Right elbow", "Right forearm",
             "Right fingers", "Left hip", "Left knee", "Left ankle", "Left toes", "Right hip", "Right knee", "Right ankle", "Right toes"]

pairs = []
for x in range(len(start_nodes)):
  pairs.append((start_nodes[x], end_nodes[x]))

In [ ]:
def df_to_matrices(df, name):
  #calculate weights of edges
  distances = []
  
  #Find distances between connected joints
  print("Distances")
  for row in range(len(df)):
    print(row)
    row_dists = []
    for x, y in pairs:
      xs, ys = get_xyz(x), get_xyz(y)
      xyz = []
      for i in range(3):
        one = df.iloc[row][xs[i]]
        two = df.iloc[row][ys[i]]
        xyz.append(one - two)
      dist = math.sqrt(xyz[0]**2 + xyz[1]**2 + xyz[2]**2)
      row_dists.append(dist)
    distances.append(row_dists)

  distances = np.asarray(distances)

  # Calculate average of distances
  print("avg Distances")
  avg_distances = []

  for x in range(19):
    avg_distances.append(np.sum(distances[:, x])/len(df))

  #Make weights inverse to average distance
  weights = []
  for x in avg_distances:
    weights.append(1/x)
  
  #Create adjacency matrix when joints are connected.
  adj_matrix= []
  print("adj matrix")
  for x in ntu_joints:
    row = []
    for y in ntu_joints:
      if (x,y) in pairs:
        row.append(1)
      else:
        row.append(0)
    adj_matrix.append(row)
  
  #Same as previous however 1 is replaced with weight of edge
  weight_adj_matrix = []
  print("wadj matrix")
  for x in ntu_joints:
    row = []
    for y in ntu_joints:
      if (x,y) in pairs:
        index = pairs.index((x,y))
        w = weights[index]
        # Clean infs out
        if (w == math.inf or w == -math.inf):
          row.append(0)
        else:
          row.append(weights[index])
      else:
        row.append(0)
    weight_adj_matrix.append(row)

  with open('/content/drive/My Drive/Colab Notebooks/Datasets/' + name + '_adj_matr.pickle', 'wb') as output:
      pickle.dump(adj_matrix, output)

  with open('/content/drive/My Drive/Colab Notebooks/Datasets/' + name + '_wadj_matr.pickle', 'wb') as output:
      pickle.dump(weight_adj_matrix, output)

In [ ]:
df_to_matrices(mhad_df, 'mhad30fps')

Streaming output truncated to the last 5000 lines.
5623
5624
5625
5626
5627
5628
5629
5630
5631
5632
5633
5634
5635
5636
5637
5638
5639
5640
5641
5642
5643
5644
5645
5646
5647
5648
5649
5650
5651
5652
5653
5654
5655
5656
5657
5658
5659
5660
5661
5662
5663
5664
5665
5666
5667
5668
5669
5670
5671
5672
5673
5674
5675
5676
5677
5678
5679
5680
5681
5682
5683
5684
5685
5686
5687
5688
5689
5690
5691
5692
5693
5694
5695
5696
5697
5698
5699
5700
5701
5702
5703
5704
5705
5706
5707
5708
5709
5710
5711
5712
5713
5714
5715
5716
5717
5718
5719
5720
5721
5722
5723
5724
5725
5726
5727
5728
5729
5730
5731
5732
5733
5734
5735
5736
5737
5738
5739
5740
5741
5742
5743
5744
5745
5746
5747
5748
5749
5750
5751
5752
5753
5754
5755
5756
5757
5758
5759
5760
5761
5762
5763
5764
5765
5766
5767
5768
5769
5770
5771
5772
5773
5774
5775
5776
5777
5778
5779
5780
5781
5782
5783
5784
5785
5786
5787
5788
5789
5790
5791
5792
5793
5794
5795
5796
5797
5798
5799
5800
5801
5802
5803
5804
5805
5806
5807
5808
5809
5810
5811
5812

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in double_scalars


In [ ]:
mhad_df.isnull().any().any()

False

In [ ]:
df_to_matrices(ntu_df, 'ntu30fps')

Streaming output truncated to the last 5000 lines.
78523
78524
78525
78526
78527
78528
78529
78530
78531
78532
78533
78534
78535
78536
78537
78538
78539
78540
78541
78542
78543
78544
78545
78546
78547
78548
78549
78550
78551
78552
78553
78554
78555
78556
78557
78558
78559
78560
78561
78562
78563
78564
78565
78566
78567
78568
78569
78570
78571
78572
78573
78574
78575
78576
78577
78578
78579
78580
78581
78582
78583
78584
78585
78586
78587
78588
78589
78590
78591
78592
78593
78594
78595
78596
78597
78598
78599
78600
78601
78602
78603
78604
78605
78606
78607
78608
78609
78610
78611
78612
78613
78614
78615
78616
78617
78618
78619
78620
78621
78622
78623
78624
78625
78626
78627
78628
78629
78630
78631
78632
78633
78634
78635
78636
78637
78638
78639
78640
78641
78642
78643
78644
78645
78646
78647
78648
78649
78650
78651
78652
78653
78654
78655
78656
78657
78658
78659
78660
78661
78662
78663
78664
78665
78666
78667
78668
78669
78670
78671
78672
78673
78674
78675
78676
78677
78678
78679
78680
7

#Graphs from matrices

In [ ]:
#save matrices of both datasets
with open('/content/drive/My Drive/Colab Notebooks/Datasets/mhad30fps_adj_matr.pickle', 'rb') as data:
    mhad_adj_matr = pickle.load(data)

with open('/content/drive/My Drive/Colab Notebooks/Datasets/mhad30fps_wadj_matr.pickle', 'rb') as data:
    mhad_wadj_matr = pickle.load(data)

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Datasets/ntu30fps_adj_matr.pickle', 'rb') as data:
    ntu_adj_matr = pickle.load(data)

with open('/content/drive/My Drive/Colab Notebooks/Datasets/ntu30fps_wadj_matr.pickle', 'rb') as data:
    ntu_wadj_matr = pickle.load(data)

In [ ]:
# Graph connection matrix, all 0's apart from diagonal 1's top left to bottom right
graph_connect_matrix = []
for x in range(20):
  row = []
  for y in range(20):
    if (x == y):
      row.append(1)
    else:
      row.append(0)
  graph_connect_matrix.append(row)

print(graph_connect_matrix)

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 

In [ ]:
# Graph connection matrix, all 0's apart from diagonaltop left to bottom right is time between frames
def make_weight_connect_matr(val):
  weight_connect_matrix = []
  for x in range(20):
    row = []
    for y in range(20):
      if (x == y):
        row.append(val)
      else:
        row.append(0)
    weight_connect_matrix.append(row)
  return weight_connect_matrix

# Convert weight adj matrix to a 30 frame weight adj matrix
def make_wadj_matrix30(matrix, weights):
  og_matrix = copy.deepcopy(matrix)
  for x in range(29):
    row_len = (2+x)*20
    new_arr = np.zeros((row_len,row_len), dtype=int)
    adj_insert = (1+x)*20
    new_arr[adj_insert:row_len, adj_insert:row_len] = matrix
    connect_insert = x*20
    weight_matrix = make_weight_connect_matr(weights[x])
    new_arr[connect_insert:adj_insert, adj_insert:row_len] = weight_matrix
    new_arr[0:len(og_matrix), 0:len(og_matrix)] = og_matrix
    og_matrix = new_arr
  return og_matrix

In [ ]:
# Convert adjacency matrix to a 30 frame adjacency matrix
def make_adj_matrix30(matrix):
  og_matrix = copy.deepcopy(matrix)
  for x in range(29):
    row_len = (2+x)*20
    new_arr = np.zeros((row_len,row_len), dtype=int)
    adj_insert = (1+x)*20
    new_arr[adj_insert:row_len, adj_insert:row_len] = matrix
    connect_insert = x*20
    new_arr[connect_insert:adj_insert, adj_insert:row_len] = graph_connect_matrix
    new_arr[0:len(og_matrix), 0:len(og_matrix)] = og_matrix
    og_matrix = new_arr
  return og_matrix

In [ ]:
# Custom spektral dataset
class MyDataset(spektral.data.dataset.Dataset):
    def __init__(self, adj_matrix, wadj_matrix, df, **kwargs):
        self.adj_matrix = adj_matrix
        self.wadj_matrix = wadj_matrix
        self.df = df
        super().__init__(**kwargs)

    def read(self):
      #Read in data from dataframe passed in
      #Dataframe becomes the (600,3) node data
      output = []

      for row in range(0, len(self.df), 30):
        label = self.df.iloc[row]['Label']
        print("row label: "+ str(row) + ", " + str(label))
        nodes_data = []
        time_weights = []

        for row_between in range(row, row+30):
          if(row_between != row):
            time_weights.append(self.df.iloc[row_between]['Time'] - self.df.iloc[row_between-1]['Time'])
          for joint in range(20):
            node_d = []
            for xyz in range(3):
              n = 1+(joint*3 + xyz)
              node_d.append(self.df.iloc[row_between][n])
            nodes_data.append(node_d)

        #make weight adj matrix
        new_wadj_matrix = make_wadj_matrix30(self.wadj_matrix, time_weights)
        nodes_data = np.asarray(nodes_data)
        adj_matrix = np.asarray(self.adj_matrix)
        weight_adj_matrix = np.asarray(new_wadj_matrix).reshape(600, 600, 1)
        label = np.array([label])

        graph = spektral.data.graph.Graph(x = nodes_data, a = adj_matrix, e = weight_adj_matrix, y = label)
        output.append(graph)
      return output

In [ ]:
adj_matrix30 = make_adj_matrix30(mhad_adj_matr)

NameError: ignored

In [ ]:
mhad_dataset = MyDataset(adj_matrix30, mhad_wadj_matr, mhad_df)

row label: 0, 1.0
row label: 30, 1.0
row label: 60, 1.0
row label: 90, 1.0
row label: 120, 1.0
row label: 150, 1.0
row label: 180, 1.0
row label: 210, 1.0
row label: 240, 1.0
row label: 270, 1.0
row label: 300, 1.0
row label: 330, 1.0
row label: 360, 1.0
row label: 390, 1.0
row label: 420, 1.0
row label: 450, 1.0
row label: 480, 1.0
row label: 510, 1.0
row label: 540, 1.0
row label: 570, 1.0
row label: 600, 1.0
row label: 630, 1.0
row label: 660, 1.0
row label: 690, 1.0
row label: 720, 1.0
row label: 750, 1.0
row label: 780, 1.0
row label: 810, 1.0
row label: 840, 1.0
row label: 870, 1.0
row label: 900, 1.0
row label: 930, 1.0
row label: 960, 1.0
row label: 990, 1.0
row label: 1020, 1.0
row label: 1050, 1.0
row label: 1080, 1.0
row label: 1110, 1.0
row label: 1140, 1.0
row label: 1170, 1.0
row label: 1200, 1.0
row label: 1230, 1.0
row label: 1260, 1.0
row label: 1290, 1.0
row label: 1320, 1.0
row label: 1350, 1.0
row label: 1380, 1.0
row label: 1410, 1.0
row label: 1440, 1.0
row label:

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Datasets/mhad_graphs30.pickle', 'wb') as output:
  pickle.dump(mhad_dataset, output)

In [ ]:
#mhad_dataset.n_graphs

In [ ]:
ntu_dataset = MyDataset(adj_matrix30, ntu_wadj_matr, ntu_df)

row label: 0, 2.0
row label: 30, 2.0
row label: 60, 2.0
row label: 90, 2.0
row label: 120, 2.0
row label: 150, 2.0
row label: 180, 2.0
row label: 210, 2.0
row label: 240, 2.0
row label: 270, 2.0
row label: 300, 2.0
row label: 330, 2.0
row label: 360, 2.0
row label: 390, 2.0
row label: 420, 2.0
row label: 450, 2.0
row label: 480, 2.0
row label: 510, 2.0
row label: 540, 2.0
row label: 570, 2.0
row label: 600, 2.0
row label: 630, 2.0
row label: 660, 2.0
row label: 690, 2.0
row label: 720, 2.0
row label: 750, 2.0
row label: 780, 2.0
row label: 810, 2.0
row label: 840, 2.0
row label: 870, 2.0
row label: 900, 2.0
row label: 930, 2.0
row label: 960, 2.0
row label: 990, 2.0
row label: 1020, 2.0
row label: 1050, 2.0
row label: 1080, 2.0
row label: 1110, 2.0
row label: 1140, 2.0
row label: 1170, 2.0
row label: 1200, 2.0
row label: 1230, 2.0
row label: 1260, 2.0
row label: 1290, 2.0
row label: 1320, 2.0
row label: 1350, 2.0
row label: 1380, 2.0
row label: 1410, 2.0
row label: 1440, 2.0
row label:

In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/Datasets/ntu_graphs30.pickle', 'wb') as output:
  pickle.dump(ntu_dataset, output)

# joblib.dump(ntu_dataset, '/content/drive/My Drive/Colab Notebooks/Datasets/ntu_graphs30.pickle')